<a href="https://www.kaggle.com/code/suyash45/yolov8-football-train-and-predict?scriptVersionId=185476508" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/footballyolov8/fr_f_70.txt
/kaggle/input/footballyolov8/fre_f_155_we.txt
/kaggle/input/footballyolov8/22d_f_165_fr.txt
/kaggle/input/footballyolov8/fr_f_130.txt
/kaggle/input/footballyolov8/fr_f_160_fre.txt
/kaggle/input/footballyolov8/fr_f_100_we.jpg
/kaggle/input/footballyolov8/fre_f_190_22d.jpg
/kaggle/input/footballyolov8/we_f_100.txt
/kaggle/input/footballyolov8/22d_f_165_fr.jpg
/kaggle/input/footballyolov8/22d_f_85_fr.txt
/kaggle/input/footballyolov8/6ty_f_110_54r.txt
/kaggle/input/footballyolov8/we_f_175_we.jpg
/kaggle/input/footballyolov8/rt3_f_20_54r.jpg
/kaggle/input/footballyolov8/54r_f_75_we.txt
/kaggle/input/footballyolov8/22d_f_15_rt3.txt
/kaggle/input/footballyolov8/fr_f_55.jpg
/kaggle/input/footballyolov8/54r_f_75.txt
/kaggle/input/footballyolov8/rt3_f_110.jpg
/kaggle/input/footballyolov8/we_f_115_fre.txt
/kaggle/input/footballyolov8/22d_f_185_6ty.txt
/kaggle/input/footballyolov8/54r_f_5_22d.jpg
/kaggle/input/footballyolov8/fre_f_45_22d.jpg
/kaggle/input/f

In [11]:
!rm -rf

In [12]:
!pip install ultralytics

In [13]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from shutil import copyfile
import matplotlib.pyplot as plt
from PIL import Image
import yaml
from ultralytics import YOLO

In [14]:
ipaths = []
tpaths = []
for dirname, _, filenames in os.walk('/kaggle/input/footballyolov8'):
    for filename in filenames:
        if filename.endswith('.jpg'):
            ipaths.append(os.path.join(dirname, filename))
        elif filename.endswith('.txt'):
            tpaths.append(os.path.join(dirname, filename))
ipaths = sorted(ipaths)
tpaths = sorted(tpaths)
print(ipaths[0])
print(len(ipaths))

/kaggle/input/footballyolov8/22d_f_120_6ty.jpg
66


In [15]:
boxdata = []
boxfile = []
for file in tpaths:
    data = np.loadtxt(file)
    boxdata.append(data)
    boxfile.append(os.path.splitext(os.path.basename(file))[0])
print(boxdata[0])

[[          0      0.2849     0.21019    0.021875    0.087963]
 [          0     0.41875     0.27546    0.017188    0.097222]
 [          0     0.32904     0.35486    0.022135     0.10694]
 [          0     0.38997     0.50671    0.032031    0.099537]
 [          0     0.42513      0.4794     0.03151    0.099537]
 [          0      0.4849     0.41412    0.034896    0.090278]
 [          0     0.48398     0.52523    0.036719    0.091204]
 [          0     0.47721     0.58102    0.048698    0.081481]
 [          1     0.64805     0.45532     0.01276    0.022685]
 [          0     0.56875     0.21736    0.019271    0.078241]
 [          0     0.58646     0.19745    0.021354    0.077315]
 [          0     0.68919     0.40833    0.033594    0.097222]
 [          0     0.95065     0.38518    0.029948     0.07963]]


In [16]:
BOX = pd.DataFrame()
for i, data in enumerate(boxdata):
    add = pd.DataFrame(data if data.ndim > 1 else [data])
    add[5] = boxfile[i]
    BOX = pd.concat([BOX, add], ignore_index=True)
BOX.columns = [0, 1, 2, 3, 4, 'file']
display(BOX)

,0,1,2,3,4,file
0,0.0,0.284896,0.210185,0.021875,0.087963,22d_f_120_6ty
1,0.0,0.418750,0.275463,0.017188,0.097222,22d_f_120_6ty
2,0.0,0.329036,0.354861,0.022135,0.106944,22d_f_120_6ty
3,0.0,0.389974,0.506713,0.032031,0.099537,22d_f_120_6ty
4,0.0,0.425130,0.479398,0.031510,0.099537,22d_f_120_6ty
...,...,...,...,...,...,...
745,0.0,0.501563,0.500926,0.047917,0.239815,we_f_50
746,0.0,0.453125,0.530556,0.035417,0.159259,we_f_50
747,0.0,0.404427,0.510417,0.070312,0.213426,we_f_50
748,0.0,0.384375,0.550231,0.018229,0.110648,we_f_50


In [17]:
class_map = {0: 'player', 1: 'football'}
BOX[0] = BOX[0].map(class_map)
display(BOX)
display(BOX[0].value_counts())

,0,1,2,3,4,file
0,player,0.284896,0.210185,0.021875,0.087963,22d_f_120_6ty
1,player,0.418750,0.275463,0.017188,0.097222,22d_f_120_6ty
2,player,0.329036,0.354861,0.022135,0.106944,22d_f_120_6ty
3,player,0.389974,0.506713,0.032031,0.099537,22d_f_120_6ty
4,player,0.425130,0.479398,0.031510,0.099537,22d_f_120_6ty
...,...,...,...,...,...,...
745,player,0.501563,0.500926,0.047917,0.239815,we_f_50
746,player,0.453125,0.530556,0.035417,0.159259,we_f_50
747,player,0.404427,0.510417,0.070312,0.213426,we_f_50
748,player,0.384375,0.550231,0.018229,0.110648,we_f_50


0
player      701
football     49
Name: count, dtype: int64

In [18]:
def draw_box(n0):
    ipath = ipaths[n0]
    image = cv2.imread(ipath)
    H, W = image.shape[:2]
    file = os.path.splitext(os.path.basename(ipath))[0]
    box = BOX[BOX['file'] == file].reset_index(drop=True)
    for _, row in box.iterrows():
        label, x, y, w, h = row[0], row[1], row[2], row[3], row[4]
        x0, y0 = int((x - w / 2) * W), int((y - h / 2) * H)
        x1, y1 = int((x + w / 2) * W), int((y + h / 2) * H)
        color = (0, 255, 0) if label == 'player' else (0, 0, 255)
        cv2.rectangle(image, (x0, y0), (x1, y1), color, 2)
    return image

In [ ]:
from matplotlib import animation, rc
rc('animation', html='jshtml')

def create_animation(ims):
    fig = plt.figure(figsize=(10, 6))
    im = plt.imshow(cv2.cvtColor(ims[0], cv2.COLOR_BGR2RGB))
    plt.close()

    def animate_func(i):
        im.set_array(cv2.cvtColor(ims[i], cv2.COLOR_BGR2RGB))
        return [im]
    return animation.FuncAnimation(fig, animate_func, frames=len(ims), interval=500)

images1 = [draw_box(i) for i in tqdm(range(len(ipaths)))]
create_animation(images1)

100%|██████████| 66/66 [00:03<00:00, 16.78it/s]


In [ ]:
os.makedirs('datasets/train', exist_ok=True)
os.makedirs('datasets/valid', exist_ok=True)
os.makedirs('datasets/test', exist_ok=True)

for i, ipath in enumerate(ipaths):
    tpath = tpaths[i]
    ifile, tfile = os.path.basename(ipath), os.path.basename(tpath)
    if i % 3 == 0:
        copyfile(ipath, f'datasets/train/{ifile}')
        copyfile(tpath, f'datasets/train/{tfile}')
    elif i % 3 == 1:
        copyfile(ipath, f'datasets/valid/{ifile}')
        copyfile(tpath, f'datasets/valid/{tfile}')
    else:
        copyfile(ipath, f'datasets/test/{ifile}')
        copyfile(tpath, f'datasets/test/{tfile}')

In [ ]:
football_yaml = dict(
    train='datasets/train',
    val='datasets/valid',
    test='datasets/test',
    nc=2,
    names=['player', 'football']
)

with open('football.yaml', 'w') as outfile:
    yaml.dump(football_yaml, outfile, default_flow_style=True)

%cat football.yaml

In [ ]:
model = YOLO("yolov8n.pt")
!yolo task=detect mode=train model=yolov8n.pt data=football.yaml epochs=20 imgsz=640